In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
dvfs_list = ['1', '0c00', '0d00', '0e00', 
             '0f00', '1000', '1100', '1200', 
             '1300', '1400', '1500', '1600', '1700', '1800', '1900', '1a00']
dvfs_dict = {
    "0x0c00" :  1.2,
    "0x0d00" :  1.3,
    "0x0e00" :  1.4,
    "0x0f00" :  1.5,
    "0x1000" : 1.6,
    "0x1100" : 1.7,
    "0x1200" : 1.8,
    "0x1300" : 1.9,
    "0x1400" : 2.0,
    "0x1500" : 2.1,
    "0x1600" : 2.2,
    "0x1700" : 2.3,
    "0x1800" : 2.4,
    "0x1900" : 2.5,
    "0x1a00" : 2.6,
    "0x1b00" : 2.7,
    "0x1c00" : 2.8,
    "0x1d00" : 2.9,
}

df_dict = {
    'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 
    
    'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 
    'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
    'erxPackets': [], 'erxBytes':[], 'etxPackets': [], 'etxBytes':[],
    
    'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 
    'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 
    'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 
    'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 
    'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 

    'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 
    'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 
    'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 
    'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 
    'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 

    'MappernumRecordsInPerSecond_avg': [], 
    'MappernumRecordsInPerSecond_std': [], 'MappernumRecordsOutPerSecond_avg': [], 
    'MappernumRecordsOutPerSecond_std': [], 'MapperbusyTimeMsPerSecond_avg': [], 
    'MapperbusyTimeMsPerSecond_std': [], 'MapperbackPressuredTimeMsPerSecond_avg': [], 
    'MapperbackPressuredTimeMsPerSecond_std': [],
    'MapperbusyTime_%': [], 'MapperbackPressuredTime_%': []
}

print(df_dict)
print("*****************************************************************")

def parseFile(rate, itr, dvfs, policy, i):
    file=f"/home/han/github/logs/static/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/summary.csv"
    print(file)
    df_dict['i'].append(i)
    df_dict['itr'].append(itr)
    if '0x'+dvfs in dvfs_dict:
        df_dict['dvfs'].append(dvfs_dict['0x'+dvfs])
    else:
        df_dict['dvfs'].append(dvfs)
    df_dict['policy'].append(policy)
    df_dict['rate'].append(rate)
        
    df = pd.read_csv(file)
    df = df[df.columns.drop(list(df.filter(regex='Cnt')))]
    df = df[df.columns.drop(list(df.filter(regex='Bytes')))]
    #print(df)
    
    dff = df[df['name'].str.contains('Sink')]
    dff.columns = 'Sink' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Source')]
    dff.columns = 'Source' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Mapper')]
    dff.columns = 'Mapper' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    jfile = f"/home/han/github/logs/static/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/server2_rapl.log"
    with open(jfile) as file:
        lines = [float(line.rstrip()) for line in file]
        print(len(lines))
        df_dict['watts_avg'].append(float(round(np.mean(lines[300:600]), 2)))
        df_dict['watts_std'].append(float(round(np.std(lines[300:600]), 2)))
        #print(np.std(lines[300:500]))
        
    jfile = f"/home/han/github/logs/static/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/stats.csv"
    with open(jfile) as file:
        poll = []
        c1 = []
        c1e = []
        c3 = []
        c6 = []
        rxp = []
        rxb = []
        txp = []
        txb = []
        erxp = []
        erxb = []
        etxp = []
        etxb = []
        for line in file:
            ll = [int(a) for a in line.strip().split(',')]
            poll.append(ll[0])
            c1.append(ll[1])
            c1e.append(ll[2])
            c3.append(ll[3])
            c6.append(ll[4])
            rxp.append(ll[5])
            rxb.append(ll[6])
            txp.append(ll[7])
            txb.append(ll[8])
            erxp.append(ll[9])
            erxb.append(ll[10])
            etxp.append(ll[11])
            etxb.append(ll[12])
        ss = 30
        ee = 60
        df_dict['pollCnt'].append(np.sum(poll[ss:ee]))
        df_dict['c1Cnt'].append(np.sum(c1[ss:ee]))
        df_dict['c1eCnt'].append(np.sum(c1e[ss:ee]))
        df_dict['c3Cnt'].append(np.sum(c3[ss:ee]))
        df_dict['c6Cnt'].append(np.sum(c6[ss:ee]))
        #'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
        df_dict['rxPackets'].append(np.sum(rxp[ss:ee]))
        df_dict['rxBytes'].append(np.sum(rxb[ss:ee]))        
        df_dict['txPackets'].append(np.sum(txp[ss:ee]))
        df_dict['txBytes'].append(np.sum(txb[ss:ee]))
        df_dict['erxPackets'].append(np.sum(erxp[ss:ee]))
        df_dict['erxBytes'].append(np.sum(erxb[ss:ee]))
        #print(np.mean(erxb[30:50]), np.std(erxb[30:50]))
        df_dict['etxPackets'].append(np.sum(etxb[ss:ee]))
        df_dict['etxBytes'].append(np.sum(etxb[ss:ee]))

{'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [], 'c3Cnt': [], 'c6Cnt': [], 'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [], 'erxPackets': [], 'erxBytes': [], 'etxPackets': [], 'etxBytes': [], 'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 'SourcebusyTime_%': []

In [4]:
#itr=1
itrs = [2, 100, 200, 400, 600]
mdvfs=["0c00", "0d00", "0e00", "0f00", "1000", "1100", "1200", "1300", "1400", "1500", "1600", "1700", "1800", "1900", "1a00"]
#mdvfs=["0c00", "1a00"]
nrepeat = 3

for itr in itrs:
    for dvfs in mdvfs:
        for qps in [100000, 200000, 300000]:
            for i in range(nrepeat):
                parseFile(qps, itr, dvfs, "userspace", i)
#print(df_dict)

/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat0/summary.csv
718
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat1/summary.csv
720
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat2/summary.csv
721
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat0/summary.csv
717
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat1/summary.csv
723
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat2/summary.csv
726
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat0/summary.csv
724
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr2_userspacedvfs0c00_repeat1/summary.csv
722
/home/han/github/logs/static/query1_cores16_frate300000_

711
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr2_userspacedvfs1400_repeat2/summary.csv
709
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr2_userspacedvfs1400_repeat0/summary.csv
712
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr2_userspacedvfs1400_repeat1/summary.csv
712
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr2_userspacedvfs1400_repeat2/summary.csv
708
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr2_userspacedvfs1400_repeat0/summary.csv
715
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr2_userspacedvfs1400_repeat1/summary.csv
710
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr2_userspacedvfs1400_repeat2/summary.csv
709
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr2_userspacedvfs1500_repeat0/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate100

718
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr100_userspacedvfs0d00_repeat1/summary.csv
717
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr100_userspacedvfs0d00_repeat2/summary.csv
719
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr100_userspacedvfs0d00_repeat0/summary.csv
718
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr100_userspacedvfs0d00_repeat1/summary.csv
716
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr100_userspacedvfs0d00_repeat2/summary.csv
719
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr100_userspacedvfs0e00_repeat0/summary.csv
715
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr100_userspacedvfs0e00_repeat1/summary.csv
715
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr100_userspacedvfs0e00_repeat2/summary.csv
716
/home/han/github/logs/static/query1_

707
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr100_userspacedvfs1500_repeat1/summary.csv
709
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr100_userspacedvfs1500_repeat2/summary.csv
710
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr100_userspacedvfs1500_repeat0/summary.csv
712
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr100_userspacedvfs1500_repeat1/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr100_userspacedvfs1500_repeat2/summary.csv
713
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr100_userspacedvfs1600_repeat0/summary.csv
712
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr100_userspacedvfs1600_repeat1/summary.csv
708
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr100_userspacedvfs1600_repeat2/summary.csv
708
/home/han/github/logs/static/query1_

719
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr200_userspacedvfs0e00_repeat0/summary.csv
718
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr200_userspacedvfs0e00_repeat1/summary.csv
718
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr200_userspacedvfs0e00_repeat2/summary.csv
724
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr200_userspacedvfs0f00_repeat0/summary.csv
717
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr200_userspacedvfs0f00_repeat1/summary.csv
722
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr200_userspacedvfs0f00_repeat2/summary.csv
719
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr200_userspacedvfs0f00_repeat0/summary.csv
719
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr200_userspacedvfs0f00_repeat1/summary.csv
718
/home/han/github/logs/static/query1_

706
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr200_userspacedvfs1600_repeat0/summary.csv
709
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr200_userspacedvfs1600_repeat1/summary.csv
711
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr200_userspacedvfs1600_repeat2/summary.csv
709
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr200_userspacedvfs1700_repeat0/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr200_userspacedvfs1700_repeat1/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr200_userspacedvfs1700_repeat2/summary.csv
705
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr200_userspacedvfs1700_repeat0/summary.csv
706
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr200_userspacedvfs1700_repeat1/summary.csv
710
/home/han/github/logs/static/query1_

717
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr400_userspacedvfs1000_repeat0/summary.csv
717
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr400_userspacedvfs1000_repeat1/summary.csv
715
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr400_userspacedvfs1000_repeat2/summary.csv
719
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr400_userspacedvfs1000_repeat0/summary.csv
715
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr400_userspacedvfs1000_repeat1/summary.csv
718
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr400_userspacedvfs1000_repeat2/summary.csv
714
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr400_userspacedvfs1000_repeat0/summary.csv
717
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr400_userspacedvfs1000_repeat1/summary.csv
720
/home/han/github/logs/static/query1_

703
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr400_userspacedvfs1800_repeat0/summary.csv
706
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr400_userspacedvfs1800_repeat1/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr400_userspacedvfs1800_repeat2/summary.csv
712
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr400_userspacedvfs1800_repeat0/summary.csv
708
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr400_userspacedvfs1800_repeat1/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr400_userspacedvfs1800_repeat2/summary.csv
710
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr400_userspacedvfs1800_repeat0/summary.csv
711
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr400_userspacedvfs1800_repeat1/summary.csv
707
/home/han/github/logs/static/query1_

712
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr600_userspacedvfs1100_repeat0/summary.csv
714
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr600_userspacedvfs1100_repeat1/summary.csv
714
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr600_userspacedvfs1100_repeat2/summary.csv
709
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr600_userspacedvfs1100_repeat0/summary.csv
714
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr600_userspacedvfs1100_repeat1/summary.csv
713
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr600_userspacedvfs1100_repeat2/summary.csv
719
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr600_userspacedvfs1200_repeat0/summary.csv
714
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr600_userspacedvfs1200_repeat1/summary.csv
717
/home/han/github/logs/static/query1_

709
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr600_userspacedvfs1900_repeat0/summary.csv
705
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr600_userspacedvfs1900_repeat1/summary.csv
704
/home/han/github/logs/static/query1_cores16_frate300000_600000_fbuff-1_itr600_userspacedvfs1900_repeat2/summary.csv
704
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr600_userspacedvfs1a00_repeat0/summary.csv
706
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr600_userspacedvfs1a00_repeat1/summary.csv
704
/home/han/github/logs/static/query1_cores16_frate100000_600000_fbuff-1_itr600_userspacedvfs1a00_repeat2/summary.csv
708
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr600_userspacedvfs1a00_repeat0/summary.csv
707
/home/han/github/logs/static/query1_cores16_frate200000_600000_fbuff-1_itr600_userspacedvfs1a00_repeat1/summary.csv
704
/home/han/github/logs/static/query1_

In [5]:
d = pd.DataFrame(df_dict)
print(d.columns)
#d0 = d[(d.itr == 1.2)]

Index(['i', 'itr', 'dvfs', 'rate', 'policy', 'watts_avg', 'watts_std',
       'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'erxPackets', 'erxBytes', 'etxPackets',
       'etxBytes', 'SinknumRecordsInPerSecond_avg',
       'SinknumRecordsInPerSecond_std', 'SinknumRecordsOutPerSecond_avg',
       'SinknumRecordsOutPerSecond_std', 'SinkbusyTimeMsPerSecond_avg',
       'SinkbusyTimeMsPerSecond_std', 'SinkbackPressuredTimeMsPerSecond_avg',
       'SinkbackPressuredTimeMsPerSecond_std', 'SinkbusyTime_%',
       'SinkbackPressuredTime_%', 'SourcenumRecordsInPerSecond_avg',
       'SourcenumRecordsInPerSecond_std', 'SourcenumRecordsOutPerSecond_avg',
       'SourcenumRecordsOutPerSecond_std', 'SourcebusyTimeMsPerSecond_avg',
       'SourcebusyTimeMsPerSecond_std',
       'SourcebackPressuredTimeMsPerSecond_avg',
       'SourcebackPressuredTimeMsPerSecond_std', 'SourcebusyTime_%',
       'SourcebackPressuredTime_%', 'MappernumRecordsInPe

##

In [34]:
d[((d.i==0) & (d.rate==300000))][['dvfs']]

,dvfs
6,1.2
15,1.3
24,1.4
33,1.5
42,1.6
...,...
636,2.2
645,2.3
654,2.4
663,2.5


In [9]:
d[((d.i==0) & (d.dvfs == 1.6) & (d.rate==300000))][['i', 'rate', 'itr', 'dvfs', 
   'SourcenumRecordsOutPerSecond_avg', 'SourcebackPressuredTimeMsPerSecond_avg', 'MapperbackPressuredTimeMsPerSecond_avg',
   'watts_avg', 'watts_std']]

,i,rate,itr,dvfs,SourcenumRecordsOutPerSecond_avg,SourcebackPressuredTimeMsPerSecond_avg,MapperbackPressuredTimeMsPerSecond_avg,watts_avg,watts_std
42,0,300000,2,1.6,266085.938056,434.333333,0.000000,76.50,0.48
177,0,300000,100,1.6,296760.144087,352.647619,0.004167,81.74,0.30
312,0,300000,200,1.6,284388.254841,396.195238,0.000000,78.19,0.31
447,0,300000,400,1.6,299983.695198,228.876190,0.016667,78.27,0.37
582,0,300000,600,1.6,279105.896706,420.397619,0.012500,80.70,1.00


In [91]:
d[((d.i==2) & (d.dvfs == 2.6) & (d.rate==200000))][['i', 'rate', 'itr', 'dvfs', 
   'SourcenumRecordsOutPerSecond_avg', 'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt',
   'watts_avg', 'watts_std', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'erxPackets', 'erxBytes', 'etxPackets',
       'etxBytes']]

,i,rate,itr,dvfs,SourcenumRecordsOutPerSecond_avg,pollCnt,c1Cnt,c1eCnt,c3Cnt,c6Cnt,watts_avg,watts_std,rxPackets,rxBytes,txPackets,txBytes,erxPackets,erxBytes,etxPackets,etxBytes
14,2,200000,2,2.6,200000.000000,9198,1628096,215075,0,1016993,68.48,1.64,48368134,70080823255,21500567,28141312531,21632,1460065,211669,211669
32,2,200000,100,2.6,200000.000000,551,548629,97057,0,478251,82.43,1.45,47939982,69535709758,20272101,27854800014,21297,1393594,97587,97587
50,2,200000,200,2.6,199995.229722,616,254087,196601,0,1014040,70.76,1.71,48028159,70155426113,19978633,28086225460,33734,2240313,221878,221878
68,2,200000,400,2.6,199989.265794,776,414358,248209,0,1351826,70.20,1.41,48239047,70073831267,19712567,28039742082,29191,1937074,184502,184502
86,2,200000,600,2.6,200000.000000,317,155313,75644,0,473585,76.71,1.56,47756896,69686342565,19164687,27855345222,30360,1969718,63229,63229


##

In [84]:
d[((d.i==0) & (d.dvfs == 2.6) & (d.rate==300000))][['i', 'rate', 'itr', 'dvfs', 
   'SourcenumRecordsOutPerSecond_avg', 'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt',
   'watts_avg', 'watts_std', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'erxPackets', 'erxBytes', 'etxPackets',
       'etxBytes']]

,i,rate,itr,dvfs,SourcenumRecordsOutPerSecond_avg,pollCnt,c1Cnt,c1eCnt,c3Cnt,c6Cnt,watts_avg,watts_std,rxPackets,rxBytes,txPackets,txBytes,erxPackets,erxBytes,etxPackets,etxBytes
15,0,300000,2,2.6,299982.753532,20364,2278184,474114,0,1663477,92.20,1.48,72493394,104950546639,33034055,42205782058,25079,1645208,115683,115683
33,0,300000,100,2.6,299992.374167,1303,2084468,406252,0,1239125,95.69,1.28,72709270,105078015159,30995728,42108730140,24153,1562321,64522,64522
51,0,300000,200,2.6,299983.228016,1218,1059353,562748,0,1485869,92.41,1.32,72367147,104596845671,30626213,41910972614,51151,3297064,76113,76113
69,0,300000,400,2.6,300005.063849,962,758822,353415,0,1328697,95.06,1.26,72125649,104926275980,29375180,41968450203,50227,3246198,73994,73994
87,0,300000,600,2.6,298217.294127,982,549648,346635,0,1809486,98.48,1.08,71514483,103943874592,28919280,41552241385,45382,2962990,145968,145968


In [87]:
d[((d.i==1) & (d.dvfs == 2.6) & (d.rate==300000))][['i', 'rate', 'itr', 'dvfs', 
   'SourcenumRecordsOutPerSecond_avg', 'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt',
   'watts_avg', 'watts_std', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'erxPackets', 'erxBytes', 'etxPackets',
       'etxBytes']]

,i,rate,itr,dvfs,SourcenumRecordsOutPerSecond_avg,pollCnt,c1Cnt,c1eCnt,c3Cnt,c6Cnt,watts_avg,watts_std,rxPackets,rxBytes,txPackets,txBytes,erxPackets,erxBytes,etxPackets,etxBytes
16,1,300000,2,2.6,300007.068016,6856,2421098,429313,0,1522054,98.59,1.14,72812412,105139148503,32669201,42246745238,22820,1490782,104226,104226
34,1,300000,100,2.6,299971.517024,1252,2035087,462236,0,1330034,91.07,1.28,72520888,104733738568,30893001,41972613269,24501,1583910,57930,57930
52,1,300000,200,2.6,300050.445913,1065,962819,505445,0,1444485,89.44,1.23,72356450,104886538722,30080526,41991215979,30170,2022195,235881,235881
70,1,300000,400,2.6,299962.565992,889,712859,319779,0,1224653,92.02,1.31,72184304,105014180589,29965582,42027034998,26263,1762667,207899,207899
88,1,300000,600,2.6,299987.256944,586,117150,163581,0,1274338,88.86,1.38,71014748,104015473794,28623139,41571822267,25663,1651579,49008,49008


In [88]:
d[((d.i==2) & (d.dvfs == 2.6) & (d.rate==300000))][['i', 'rate', 'itr', 'dvfs', 
   'SourcenumRecordsOutPerSecond_avg', 'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt',
   'watts_avg', 'watts_std', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'erxPackets', 'erxBytes', 'etxPackets',
       'etxBytes']]

,i,rate,itr,dvfs,SourcenumRecordsOutPerSecond_avg,pollCnt,c1Cnt,c1eCnt,c3Cnt,c6Cnt,watts_avg,watts_std,rxPackets,rxBytes,txPackets,txBytes,erxPackets,erxBytes,etxPackets,etxBytes
17,2,300000,2,2.6,300004.447659,12183,1977817,228640,0,970412,93.99,1.45,72362889,105032970202,32306123,42183065937,28811,1858648,56525,56525
35,2,300000,100,2.6,300010.790198,1279,2154016,356081,0,1063610,89.45,1.24,71856464,103976650655,30614542,41675020142,32957,2178606,188873,188873
53,2,300000,200,2.6,299950.068135,1171,1042591,594819,0,1483605,92.68,1.17,72447457,104975494194,30943338,42082335573,34869,2267795,111726,111726
71,2,300000,400,2.6,299960.515238,1047,745512,390845,0,1456961,90.46,1.42,72369619,105125691890,30026603,42071616087,28946,1916110,179104,179104
89,2,300000,600,2.6,299989.702817,799,666421,261311,0,1232466,88.15,1.23,71796206,104683995797,28786272,41829401381,29087,1897661,110679,110679
